In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100

df_qrels = pd.read_csv("../runs/google_questions/qrels.csv")

In [2]:
df_qrels.head(2)

,country,id,query,docid,docid_text,query_date,answer_date,match_score,expanded_search,answer_type,split,id_country
0,mx,4,1 kg a cuantos miligramos equivale,1595#34,"1 ""kilogramo"" es equivalente a:",2024-04-10,2024-04-17,0.6829,False,feat_snip,train,NaN
1,mx,5,1 kilo a cuantos miligramos equivale,1595#34,"1 ""kilogramo"" es equivalente a:",2024-04-10,2024-04-17,0.6829,False,feat_snip,train,NaN


### Questions for quality assessment

In [7]:
# Consider only questions from "general", "test":
df_quality = df_qrels.query("country == 'full' & split == 'test'").copy()

In [8]:
# Add the title of the article:
import datasets

collection_dataset = datasets.load_dataset("../runs/corpora/eswiki-20240401-corpus")
collection_dataset = collection_dataset["train"].select_columns(["docid", "title"])
df_quality = df_quality.merge(collection_dataset.to_pandas(), left_on="docid", right_on="docid")
df_quality["text"] = df_quality["title"] + ". " + df_quality["docid_text"]

Resolving data files:   0%|          | 0/29 [00:00<?, ?it/s]

In [9]:
df_quality = df_quality[["id", "query", "text"]].copy()
df_quality.head(2)

,id,query,text
0,5332643,en grecia quién aplico la democracia radical,Efialtes de Atenas. Efialtes es considerado po...
1,5332652,1 bar a cuantos litros equivale,Volumen molar. Experimentalmente y mediante la...


In [13]:
df_quality = df_quality[["id", "query", "text"]].copy()
df_quality.head(2)

,id,query,text
0,14,10 ejemplos de compuestos cuaternarios,Catión de amonio cuaternario. Se ha demostrado...
1,3124584,que virus mata el amonio cuaternario,Catión de amonio cuaternario. Se ha demostrado...


In [18]:
from pathlib import Path

df_all = df_quality.sample(50, random_state=33)

outdir = "../runs/google_questions/quality_annotation"
Path(outdir).mkdir(parents=True, exist_ok=True)

annotators = ["A1", "A2", "A3", "A4"]

ids_done = df_all["id"].tolist()
dfs_annotators = {}
for annotator in annotators:
    dfs_annotators[annotator] = df_quality[~df_quality["id"].isin(ids_done)].sample(200, random_state=33)
    ids_done.extend(dfs_annotators[annotator]["id"].tolist())
    dfs_annotators[annotator] = pd.concat(
        [df_all, dfs_annotators[annotator]]).sample(frac=1, random_state=ord(annotator[0]))
    dfs_annotators[annotator].to_csv(f"{outdir}/{annotator}.csv", index=False)
